In [ ]:
from railguard.railguard import *
import google.generativeai as genai
import dotenv
import sys 
from loguru import logger

In [ ]:
''' 
logger.trace("This is a trace message.")
logger.debug("This is a debug message")
logger.info("This is an info message.")
logger.success("This is a success message.")
logger.warning("This is a warning message.")
logger.error("This is an error message.")
logger.critical("This is a critical message.")

'''

In [ ]:
RAILGUARD_MODEL = 'gemini-2.0-flash-exp'

# Save logs to a file at the specified path
LOG_FILE_PATH = 'logs/railguard.log'


In [ ]:
class ChatBot:
    def __init__(self):

        # Load environment variables
        api_key = os.getenv("GOOGLE_API_KEY")

        # Class variables
        self.question = ''
        self.answer = ''
        self.relevant_pineconedata = ''

        # Initialize logger 
        logger.remove()  
        logger.add(
            "logs/logfile_{time:YYYY-MM-DD}.log",  # Daily log file
            rotation="1 day",  # Rotate every day
            retention="7 days",  # Keep logs for 7 days
            compression="zip",  # Compress old logs
            enqueue=True  # Ensures thread-safe logging
        )
        self.logger = logger

        # Initialize Model Loader
        genai.configure(api_key=api_key)
        llm_prompt = """
        Act as an expert legal assistant with comprehensive knowledge of statutory law and case precedent. 
        Analyze the following legal question carefully, then answer user questions through rigorous legal reasoning."""

        self.chat_model = genai.GenerativeModel('gemini-1.5-flash', system_instruction=llm_prompt)
        self.logger.info("Model loaded successfully.")

        # Initialize RailGuard
        self.railguard_obj = RailGuard(model_name=RAILGUARD_MODEL)
        self.logger.success("RailGuard initialized successfully.")

    def run(self):
        while True:
            try:
                self.question = input("Enter your question: ")
                if self.question.lower() == 'exit':
                    break

                self.railguard_eval = self.railguard_obj.railguard_eval(self.question)

                self.logger.info(f"Question: {self.question}")
                self.logger.info(f"Railguard Evaluation: {self.railguard_eval}")

                if self.railguard_eval:
                    # Generate the answer
                    response = self.chat_model.generate_content(self.question)
                    self.answer = response.text  # Extract the text response

                    self.logger.info(f"Answer: {self.answer}")
                    print(f"Answer: {self.answer}")

                else:
                    self.logger.warning("Railguard evaluation failed.")
                    print("Railguard evaluation failed. Please ask a question related to F1 visa or student visa only.")

            except Exception as e:
                self.logger.error(f"An error occurred: {e}")
                print(f"An error occurred: {e}")


# Run the chatbot
obj = ChatBot()
obj.run()


In [ ]:
" \n\n[X] A\n[X] B\n[X] C\n[X]",
" \n\n[X] B\n[X] C\n[X] D\n[X]",
" [X]  (A)  (B)  (C)",
" \n\n[X]  (A)  (B)  (C)",